In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
def merge_dataframes_for_train(user_path, file_type, count, not_use_postfixes = []):
    i = 0
    df_list = []
    used_postfixes = []
    for file in os.listdir(user_path):
        if i == count:
            break
        if file.find(file_type) != -1 and file[-11:] not in not_use_postfixes:
            df_list.append(pd.read_csv(os.path.join(user_path, file)))
            used_postfixes.append(file[-11:])
            i += 1
            
    return pd.concat(df_list, ignore_index=True), used_postfixes       

In [3]:
train_df, used = merge_dataframes_for_train("..\\..\\scripts\\_feature_events\\15min\\rolling\\60s\\user_1", "bt", 4)

In [4]:
train_df['user'] = train_df['user_user_agg']

In [5]:
train_df = train_df.drop(['timestamp', 'user_user_agg'], axis=1)

In [6]:
train_df = (train_df - train_df.min()) / (train_df.max() - train_df.min())

In [7]:
def drop_bad_rows(df):
    bad_rows = set()
    for col in df.columns:
        if col != "user":
            for user in df.user.unique():
                for x in list(
                    df.loc[df.user == user, :][
                        np.abs(stats.zscore(df.loc[df.user == user, col])) > 3
                    ].index
                ):
                    bad_rows.add(x)

            for x in list(df[col][np.abs(stats.zscore(df[col])) > 3].index):
                bad_rows.add(x)

    df.drop(list(bad_rows), axis=0, inplace=True)

    return df

In [8]:
train_df = drop_bad_rows(train_df)

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  


In [9]:
test_df, _ = merge_dataframes_for_train("..\\..\\scripts\\_feature_events\\15min\\rolling\\60s\\user_1", "bt", 3, used)
test_df['user'] = test_df['user_user_agg']
test_df = test_df.drop(['timestamp', 'user_user_agg'], axis=1)
test_df = (test_df - test_df.min()) / (test_df.max() - test_df.min())

In [10]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [11]:
dataset = train_df.to_numpy().copy()
X = dataset[:, :-1]
y = dataset[:, -1]

In [12]:
C = 1
kernel = 'rbf'
degree = 1
gamma = 5

In [13]:
model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, probability=True)

In [14]:
model.fit(X, y)

SVC(C=1, degree=1, gamma=5, probability=True)

In [15]:
dataset = test_df.to_numpy().copy()
X_test = dataset[:, :-1]
y_test = dataset[:, -1]

In [16]:
y_t = y_test.copy()
predict = model.predict(X_test).copy()
predict_pr = model.predict_proba(X_test).copy()

In [17]:
accuracy_score(y_t, predict)

0.6202090592334495

In [18]:
y_t

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [19]:
f1_score(y_t, predict)

0.7655913978494624